In [1]:
import os

In [2]:
%pwd

'c:\\Users\\msi\\Documents\\projects\\ml-mlf-prj\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\msi\\Documents\\projects\\ml-mlf-prj'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pascalmillimono/ml-mlf-prj.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pascalmillimono"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5372a79d362f007179b79d84e7f386837bc29a90"

In [9]:
#---4. Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    """l1_ration, alpha => will be define in params.yaml as there are parameters
    in params.yaml => we can set params random forest, linear regression, decision tree etc...
    target will be get from schema.yaml
    """
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [27]:
#----5 Update the configuraiton manager ==> will go to src-> mlProject -> config -> configuration.py
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/pascalmillimono/ml-mlf-prj.mlflow"
        )

        return model_evaluation_config

In [14]:
#---6. Update the components --> will go into src->mlProject -> components->model_evaluation.py (create a file model_evaluation.py)
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import pandas as pd

In [30]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data  = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        # print(train_data.loc[:, 'quality']) => okay
        #print(train_data.loc[:, self.config.target_column])=> okay
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            #saving metrics as local
            scores = {"rmse": rmse, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                #Register the model
                #There are other ways to use the model registrym which depends on the use case,
                #please refer to the doc for more imformation
                #https://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
        

In [31]:
##-----7. Update the pipeline => will go in scr->mlProject -> pipeline -> stage_05_data_evaluation.py (create file stage_05_data_evaluation.py )
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-23 19:21:24,366 : INFO: common: yamll file: config\config.yaml loaded successfully]
[2024-08-23 19:21:24,368 : INFO: common: yamll file: params.yaml loaded successfully]
[2024-08-23 19:21:24,371 : INFO: common: yamll file: schema.yaml loaded successfully]
[2024-08-23 19:21:24,373 : INFO: common: created directory at: artifacts]
[2024-08-23 19:21:24,376 : INFO: common: created directory at: artifacts/model_evaluation]
[2024-08-23 19:21:24,585 : INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Successfully registered model 'ElasticnetModel'.
2024/08/23 19:21:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
